# [Conduct a Vector Similarity Search](https://milvus.io/docs/v2.3.0-beta/search.md)

> This topic describes how to search entities with Milvus.

A vector similarity search in Milvus calculates the distance between query vector(s) and vectors in the collection with specified similarity metrics, and returns the most similar results. By specifying a boolean expression that filters the scalar field or the primary key field, you can perform a hybrid search or even a search with Time Travel.

# Connect to milvus

In [1]:
#r "nuget: Milvus.Client, 2.2.1-alpha.7"
#!import config/Settings.cs

using Milvus.Client;
using Milvus.Client.Client;
using Milvus.Client.Client.gRPC;
using Milvus.Client.Client.REST;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

//Connect to milvus
(string endpoint,int port,string userName,string password) = Settings.LoadFromFile();

IMilvusClient milvusClient = default;

milvusClient = new MilvusGrpcClient(endpoint,port,userName,password);
milvusClient

Installed Packages Milvus.Client, 2.2.1-alpha.7

Address,in01-a01c84443bc084c.aws-us-west-2.vectordb.zillizcloud.com:19535


# Load collection

All search and query operations within Milvus are executed in memory. Load the collection to memory before conducting a vector similarity search.

In [5]:
string collectionName = "book";

if(!(await milvusClient.HasCollectionAsync(collectionName)))
{
    Console.WriteLine($"Collection {collectionName} not exist");
    return;
}

await milvusClient.LoadCollectionAsync(collectionName);

//Waiting for collection loaded, skip this code when you are using a MilvusRestClient
var progressItems = milvusClient.WaitForLoadingProgressCollectionValueAsync(
    collectionName,
    null,
    TimeSpan.FromMilliseconds(500),
    TimeSpan.FromSeconds(10));

await foreach(var progress in progressItems){
    Console.WriteLine($"{progress}%");
}

var collectionInfo = await milvusClient.DescribeCollectionAsync(collectionName);
collectionInfo

0%
0%
0%
100%


Milvus.Client.DetailedMilvusCollection Aliases CollectionName book CollectionId 441943051529528722 ConsistencyLevel Session CreatedUtcTime 2023-06-05 15:08:02Z Schema CollectionSchema: {AutoId: False, Description, , Fields: 4} AutoId False Description Fields index value 0 Milvus.Client.FieldType AutoId False DataType Int64 Description <null> FieldId 100 IndexParams (empty) IsPrimaryKey True Name book_id TypeParams (empty) 1 Milvus.Client.FieldType AutoId False DataType Int64 Description <null> FieldId 101 IndexParams (empty) IsPrimaryKey False Name word_count TypeParams (empty) 2 Milvus.Client.FieldType AutoId False DataType VarChar Description <null> FieldId 102 IndexParams (empty) IsPrimaryKey False Name book_name TypeParams key value max_length 256 3 Milvus.Client.FieldType AutoId False DataType FloatVector Description <null> FieldId 103 IndexParams (empty) IsPrimaryKey False Name book_intro TypeParams key value dim 2 Name book ShardsNum 1 StartPositions (empty)

# Prepare search parameters

| Parameter | Description | Options |
| --- | --- | --- |
| TopK | Number of the most similar results to return. | N/A |
| Params | Search parameter(s) specific to the index. | See [Vector Index](https://milvus.io/docs/v2.3.0-beta/index.md) for more information.

# Conduct a vector search

🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍

In [6]:
List<string> search_output_fields = new() { "book_id" };
List<List<float>> search_vectors = new() { new() { 0.1f, 0.2f } };
var searchResult = await milvusClient.SearchAsync(
    MilvusSearchParameters.Create(collectionName, "book_intro", search_output_fields)
    .WithVectors(search_vectors)
    .WithConsistencyLevel(MilvusConsistencyLevel.Strong)
    .WithMetricType(MilvusMetricType.IP)
    .WithTopK(topK: 2)
    .WithParameter("nprobe", "10")
    .WithParameter("offset", "5"));

searchResult.Results

Milvus.Client.MilvusSearchResultData FieldsData index value 0 Field: {FieldName: book_id, DataType: Int64, Data: 2, RowCount: 2} Data [ 1655, 1436 ] RowCount 2 FieldName book_id FieldId 0 DataType Int64 Ids Milvus.Client.MilvusIds IdField Milvus.Client.IdField IdFieldCase IntId IntId Milvus.Client.MilvusId`1[System.Int64] Data [ 1655, 1436 ] StrId <null> NumQueries 1 Scores [ 639963140, 632540350 ] TopK 2 TopKs [ 2 ]

| Parameter         | Description                                                  | Options                                                      |
| ----------------- | ------------------------------------------------------------ | ------------------------------------------------------------ |
| CollectionName    | Name of the collection to load.                              | N/A                                                          |
| MetricType        | Metric type used for search.                                 | This parameter must be set identical to the metric type used for index building. |
| OutFields         | Name of the field to return.                                 | Vector field is not supported in current release.            |
| Vectors           | Vectors to search with.                                      | N/A                                                          |
| VectorFieldName   | Name of the field to search on.                               | N/A                                                          |
| Expr              | Boolean expression used to filter attribute.                  | See Boolean Expression Rules for more information.           |
| ConsistencyLevel  | The consistency level used in the query.                      | STRONG, BOUNDED, and EVENTUALLY.                             |

# Release the Collection.

🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃🍃

> Release the collection loaded in Milvus to reduce memory consumption when the search is completed.

In [4]:
await milvusClient.ReleaseCollectionAsync(collectionName);

var collectionInfo = await milvusClient.DescribeCollectionAsync(collectionName);
collectionInfo

Milvus.Client.DetailedMilvusCollection Aliases CollectionName book CollectionId 441943051529528722 ConsistencyLevel Session CreatedUtcTime 2023-06-05 15:08:02Z Schema CollectionSchema: {AutoId: False, Description, , Fields: 4} AutoId False Description Fields index value 0 Milvus.Client.FieldType AutoId False DataType Int64 Description <null> FieldId 100 IndexParams (empty) IsPrimaryKey True Name book_id TypeParams (empty) 1 Milvus.Client.FieldType AutoId False DataType Int64 Description <null> FieldId 101 IndexParams (empty) IsPrimaryKey False Name word_count TypeParams (empty) 2 Milvus.Client.FieldType AutoId False DataType VarChar Description <null> FieldId 102 IndexParams (empty) IsPrimaryKey False Name book_name TypeParams key value max_length 256 3 Milvus.Client.FieldType AutoId False DataType FloatVector Description <null> FieldId 103 IndexParams (empty) IsPrimaryKey False Name book_intro TypeParams key value dim 2 Name book ShardsNum 1 StartPositions (empty)